# Non Negative Matrix Factorisation

This assignment will cover Non Negative Matrix Factorisation. Two algorithms using squared Euclidean distance and Kullback-Liebler Divergence cost functions will be built and investigated. Both algorithms will apply multiplicative updates to achieve optimisation.

The notebook will use two image datasets - CroppedYaleB and ORL. 

CroppedYaleB can be found here:
http://vision.ucsd.edu/~leekc/ExtYaleDatabase/ExtYaleB.html

and ORL here:
http://www.cl.cam.ac.uk/Research/DTG/attarchive:pub/data/att_faces.zip


In [7]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import random
import datetime

def load_data(root='C:/Users/Harrison/Documents/Uni Work/Advanced Machine Learning/data/CroppedYaleB', reduce=4):
    images, labels = [], []
    for i, person in enumerate(sorted(os.listdir(root))):
        if not os.path.isdir(os.path.join(root, person)):
            continue        
        for fname in os.listdir(os.path.join(root, person)):  
            # Remove background images in Extended YaleB dataset.
            if fname.endswith('Ambient.pgm'):
                continue            
            if not fname.endswith('.pgm'):
                continue                
            # load image.
            img = Image.open(os.path.join(root, person, fname))
            img = img.convert('L') # grey image.
            # reduce computation complexity.
            img = img.resize([s//reduce for s in img.size])
            # preprocessing (normalisation)
            img = (img-np.min(img))/(np.max(img)-np.min(img))
            # convert image to numpy array.
            img = np.asarray(img).reshape((-1,1))           
            # collect data and label.
            images.append(img)
            labels.append(i)
    # concate all images and labels.
    images = np.concatenate(images, axis=1)
    labels = np.array(labels)
    return images, labels

def load_data_without_normalisation(root='C:/Users/Harrison/Documents/Uni Work/Advanced Machine Learning/data/CroppedYaleB', reduce=4):
    images, labels = [], []
    for i, person in enumerate(sorted(os.listdir(root))):
        if not os.path.isdir(os.path.join(root, person)):
            continue        
        for fname in os.listdir(os.path.join(root, person)):  
            # Remove background images in Extended YaleB dataset.
            if fname.endswith('Ambient.pgm'):
                continue            
            if not fname.endswith('.pgm'):
                continue                
            # load image.
            img = Image.open(os.path.join(root, person, fname))
            img = img.convert('L') # grey image.
            # reduce computation complexity.
            img = img.resize([s//reduce for s in img.size])
            # preprocessing (normalisation)
            img = (img-np.min(img))/(np.max(img)-np.min(img))
            # convert image to numpy array.
            img = np.asarray(img).reshape((-1,1))           
            # collect data and label.
            images.append(img)
            labels.append(i)
    # concate all images and labels.
    images = np.concatenate(images, axis=1)
    labels = np.array(labels)
    return images, labels


The functions below are non negative matrix factorisation algorithms written using standard squared Euclidean distance and KL Divergence cost functions. The multiplicative update rules are written to gaurantee optimisation. 

In [8]:
#function for NMF using a euclidean based cost function and a multiplicative update algorithm

def NMF_euc(V, max_iterations):
    n = V.shape[0]
    m = V.shape[1]
    r = np.linalg.matrix_rank(V)
    W = np.random.rand(n, r)
    H = np.random.rand(r, m)        
    records = []
    iteration_count = 0
    conv_criteria = []
    stop_flag = 0
#stop criteria maxes at specified max_iterations or when ratio of the
#difference in loss is between 0.99 and 1 for last 20 iterations(i.e. convergence)
    while iteration_count < max_iterations and stop_flag != 1:
        W_stationary = W
        H_stationary = H
        top_H = np.dot(W_stationary.T, V)
        bottom_H = W_stationary.T.dot(np.dot(W_stationary, H_stationary))
        for i in range(H.shape[0]):
            for j in range(H.shape[1]):
                H[i][j] = H_stationary[i][j] * top_H[i][j] / bottom_H[i][j]  
        H_stationary = H
        W_stationary = W
        top_W = np.dot(V, H_stationary.T)
        bottom_W = W_stationary.dot(np.dot(H_stationary, H_stationary.T))
        for i in range(W.shape[0]):
            for j in range(W.shape[1]):
                W[i][j] = W_stationary[i][j] * top_W[i][j] / bottom_W[i][j]
        loss = np.sum((V - W.dot(H))**2)   
        records.append(loss)
        iteration_count += 1
        ##print('Iteration %s: %s' %(iteration_count, loss))
        if iteration_count > 100:
            try:
                conv_1 = abs(records[-2] - records[-1])
                conv_2 = abs(records[-3] - records[-2])
                convergence_value = conv_1/conv_2
                conv_criteria.append (convergence_value)
                last_20_1 = np.array(conv_criteria[-20:]) > 0.99 
                last_20_2 = np.array(conv_criteria[-20:]) < 1  
                if np.sum(last_20_1) == np.sum(last_20_2):
                    stop_flag = 1
                else:
                    stop_flag = 0
            except:
                next
    return W,H

#KL Divergence NMF
def NMF_div(V, max_iterations):
    n = V.shape[0]
    m = V.shape[1]
    r = np.linalg.matrix_rank(V)
    W = np.random.rand(n, r)
    H = np.random.rand(r, m)        
    records = []
    iteration_count = 0
    conv_criteria = []
    stop_flag = 0
    while iteration_count < max_iterations and stop_flag != 1:
        W_stationary = W
        H_stationary = H
        top_H = W_stationary.T.dot(V/np.dot(W_stationary, H_stationary))
        bottom_H = np.dot(W_stationary.T, np.ones(V.shape))
        for i in range(H.shape[0]):
            for j in range(H.shape[1]):
                H[i][j] = H_stationary[i][j] * top_H[i][j] / bottom_H[i][j]  
        H_stationary = H
        W_stationary = W
        top_W = np.dot((V/np.dot(W_stationary, H_stationary)), H_stationary.T)
        bottom_W = np.dot(np.ones(V.shape),H_stationary.T)
        for i in range(W.shape[0]):
            for j in range(W.shape[1]):
                W[i][j] = W_stationary[i][j] * top_W[i][j] / bottom_W[i][j]
        z = V/(np.dot(W,H))
        z[z<=0] = 0.001
        loss = np.sum(V*np.log(z) - V + W.dot(H))
        records.append(loss)
        iteration_count += 1
        #print(loss)
        if iteration_count > 100:
            try:
                conv_1 = abs(records[-2] - records[-1])
                conv_2 = abs(records[-3] - records[-2])
                convergence_value = conv_1/conv_2
                conv_criteria.append (convergence_value)
                last_20_1 = np.array(conv_criteria[-20:]) > 0.99 
                last_20_2 = np.array(conv_criteria[-20:]) < 1  
                if np.sum(last_20_1) == np.sum(last_20_2):
                    stop_flag = 1
                else:
                    stop_flag = 0
            except:
                next
    return W,H

These functions will be used to measure the robustness of each algorithm.

In [9]:
#robustness evaluation metrics to be used in the experiment

from collections import Counter
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
from sklearn.metrics import normalized_mutual_info_score

def assign_cluster_label(X, Y):
    kmeans = KMeans(n_clusters=len(set(Y))).fit(X)
    Y_pred = np.zeros(Y.shape)
    for i in set(kmeans.labels_):
        ind = kmeans.labels_ == i
        Y_pred[ind] = Counter(Y[ind]).most_common(1)[0][0] # assign label.
    return Y_pred

#kmeans = KMeans(n_clusters=len(set(Y))).fit(V_recon.T)

#Y_pred = assign_cluster_label(np.dot(W,H).T,Y)
#acc = accuracy_score(Y, Y_pred)

###Relative reconstruction errors (RRE)
#RRE = np.linalg.norm(V-np.dot(W, H)) / np.linalg.norm(V)

###NMI
#nmi = normalized_mutual_info_score(Y, Y_pred)

In [10]:
#Noise functions

def add_gaussian_noise(V, mean, std_dev):
    V_noise = np.ones(V.shape)
    for column in range(V.shape[1]):
        gaussian_noise = np.random.normal(mean, std_dev, V.shape[0])
        gaussian_noise[gaussian_noise<0] = 0
        V_noise[:,column] = V[:,column] + gaussian_noise        
    V_noise[V_noise<0] = 0
    return V_noise
        
def add_laplacian_noise(V, centre, scale):
    laplac_noise = np.random.laplace(centre,scale,V.shape)
    laplac_noise[laplac_noise<0] = 0
    V_noise = V + laplac_noise
    return V_noise

def add_sparse_noise(V, width, height, reshape_1, reshape_2):
    V_noise = []
    for image in range(V.shape[1]):
        img = V[:, image].reshape(reshape_1,reshape_2)
        start_x = np.random.randint(0, reshape_2 - width)
        start_y = np.random.randint(0, reshape_1 - height)
        img[start_y: start_y+height, start_x:start_x+width] = 1
        img = np.asarray(img).reshape((-1,1))
        V_noise.append(img)
    V_noise = np.concatenate(V_noise, axis=1)
    return V_noise

The code below can be described as three separate experiments. Each experiment will add a different type of noise to the dataset (Gaussian, Laplacian and Sparse* respectively) and measure the robustness of the algorithm against RRE, Accuracy and NMI. Within each experiment, the algorithm will be run 5 times and each metric will then be averaged and recorded.

Each code will have the optional functions commented out. Feel free to activate them as appropriate. The commented-out code will include:
1) Reading in the different datasets (ORL or YALEB)
2) Switching between Euclidean or Divergence NMF algorithms
3) Adding different types of noise (Gaussian, Laplacian and Sparse)

Please refer to report on what sparse noise looks like.

In [12]:
#load the ORL Database
V, Y = load_data(root='/Users/joshhuang/Desktop/University/Master of Data Science/COMP5328/data/ORL', reduce=3)
print('ORL dataset: X.shape = {}, Y.shape = {}'.format(V.shape, Y.shape))  #reshape numbers: 37, 30

#load the YaleB Database
#V, Y = load_data(root='C:/Users/Harrison/Documents/Uni Work/Advanced Machine Learning/data/CroppedYaleB', reduce=6)
#print('Extended YalB dataset: X.shape = {}, Y.shape = {}'.format(V.shape, Y.shape)) #reshape numbers 32, 28

rre_results = []
acc_results = []
nmi_results = []

for i in range(5):       
    subset = random.sample(range(V.shape[1]), int(0.9*V.shape[1])) #ORL dataset
    #subset = random.sample(range(V.shape[1]), int(0.15*V.shape[1])) #YaleB dataset, reduce dataset to reduce computational time
    V_test = V[:, subset]
    Y_test = Y[subset]
        
    #adding noise
    ##gaussian noise
    #V_noise = add_gaussian_noise(V_test, 0, 0.15)
    
    ##laplacian noise
    #V_noise = add_laplacian_noise(V_test, 0, 2)
    
    ##sparse noise
    V_noise = add_sparse_noise(V_test, 10, 5, 37, 30) #if ORL dataset, using reduce = 3
    #V_noise = add_sparse_noise(V_test, 10, 5, 32, 28 ) #if YaleB dataset, using reduce = 6
    
    #apply NMF
    W, H = NMF_euc(V_noise, 500)
    #W, H = NMF_div(V_noise, 500)
    
    #evaluation metrics
    RRE = np.linalg.norm(V_test-np.dot(W, H)) / np.linalg.norm(V_test)
    rre_results.append(RRE)
    
    Y_pred = assign_cluster_label(np.dot(W,H).T,Y_test)
    acc = accuracy_score(Y_test, Y_pred)
    acc_results.append(acc)
    
    nmi = normalized_mutual_info_score(Y_test, Y_pred)
    nmi_results.append(nmi)
    print(nmi)
    
#evaluate results
rre_mean = np.round(np.mean(rre_results), 5)
acc_mean = np.round(np.mean(acc_results), 5)
nmi_mean = np.round(np.mean(nmi_results), 5)
print("Results:")
print('RRE mean: %s' %(rre_mean))
print('Acc mean: %s' %(acc_mean))
print('NMI mean: %s' %(nmi_mean))

ORL dataset: X.shape = (1110, 400), Y.shape = (400,)
0.787117189859168
0.7905379756182854
0.7901003797564998
0.8023462138267551
0.8016749265455485
Results:
RRE mean: 0.1088
Acc mean: 0.655
NMI mean: 0.79436


In [ ]:
#reconstruct images with noise #ORL datasets
image_original = V_subset[:,0].reshape(37,30)
plt.imshow(image_original, cmap=plt.cm.gray)

noisy_image = V_noise[:,0].reshape(37,30)
plt.imshow(noisy_image, cmap=plt.cm.gray)

#reconstruct images with noise #YaleB datasets
image_original = V_subset[:,0].reshape(32,28) 
plt.imshow(image_original, cmap=plt.cm.gray)

noisy_image = V_noise[:,0].reshape(32,28)
plt.imshow(noisy_image, cmap=plt.cm.gray)

#applying R
r = 300
W_final_chop = W_final[:,:r]
H_final_chop = H_final[:r,:]
recon_r = np.dot(W_final_chop, H_final_chop)
noisy_image_r = recon_r[:,0].reshape(37,30)
plt.imshow(noisy_image_r, cmap=plt.cm.gray)

In [13]:
#noise sampling code
mu, sigma = 0.5, 0.16
gaussian_noise = np.random.normal(mu, sigma, V_subset.shape[0])
hist, bins = np.histogram(gaussian_noise, bins=100, normed=True)
bin_centers = (bins[1:]+bins[:-1])*0.5
plt.plot(bin_centers, hist)

cauchy_noise = np.random.standard_cauchy((V_subset.shape[0]))
cauchy_noise.shape
hist, bins = np.histogram(cauchy_noise, bins=100, normed=True)
bin_centers = (bins[1:]+bins[:-1])*0.5
plt.plot(bin_centers, hist)

laplac_noise = np.random.laplace(0, 0.2, size = (100,1))
hist, bins = np.histogram(laplac_noise, bins=100, normed=True)
bin_centers = (bins[1:]+bins[:-1])*0.5
plt.plot(bin_centers, hist)

ORL dataset: X.shape = (1110, 400), Y.shape = (400,)


## Robust L1-NMF 

The code below was our attempt to create Robust L1-NMF. This was our initial test run before we decided to implement it into a final algorithm. Unfortunately we were unable to get this to converge properly and hence have left out the results from this algorithm from our report

In [ ]:
# Load ORL dataset.
W, X = load_data(root='/Users/joshhuang/Desktop/University/Master of Data Science/COMP5328/data/ORL', reduce=2)
print('ORL dataset: X.shape = {}, Y.shape = {}'.format(W.shape, X.shape))

#input_matrix = np.abs(np.random.uniform(low=0.0,high=255.0,size=(100,100)))
input_matrix = W
S_lambda = 0.5
R = 200

## Dimensions of the matrix
rows, cols = input_matrix.shape
W_matrix = np.abs(np.random.uniform(low=0.0,high=255.0,size=(rows, R)))
H_matrix = np.abs(np.random.uniform(low=0.0,high=255.0,size=(R, cols)))

W_matrix = np.divide(W_matrix, R*W_matrix.max())
H_matrix = np.divide(H_matrix, R*H_matrix.max())

## Iterations
num_iter = 100
num_display_cost = max(int(num_iter/10), 1)

s_lambda=1.0

for i in range(num_iter):

    # Define new S matrix
    S_matrix = input_matrix - np.dot(W_matrix, H_matrix)
    
    # Regularize the S Matrix
    S_matrix = soft_thresholding_operator(S_matrix, S_lambda)
    
    # Update the W matrix
    W_matrix = (np.abs(np.dot((S_matrix - input_matrix),H_matrix.T))\
               - np.dot((S_matrix - input_matrix),H_matrix.T))\
               /(2 * np.dot(np.dot(W_matrix, H_matrix), H_matrix.T))*W_matrix
    
    # Update the H matrix
    H_matrix = (np.abs(np.dot(W_matrix.T,(S_matrix-input_matrix)))\
               - np.dot(W_matrix.T,(S_matrix-input_matrix)))\
               /(2 * np.dot(np.dot(W_matrix.T,W_matrix), H_matrix))*H_matrix
    
    # Normalize the W and H matrices
    W_matrix = W_matrix/np.sqrt(np.sum(W_matrix**2))
    H_matrix = H_matrix*np.sqrt(np.sum(W_matrix**2))
    
    # Calculate the Euclidean cost
    c = l1_euclidean_cost(input_matrix, W_matrix, H_matrix, S_matrix)
    if i%num_display_cost==0:
        print ('Iteration',i,':', c)